In [1]:
import pandas as pd
import os

input_dir = "00_Input"
output_dir = "01_output"

# 
### LOAD NETECH DATA
# 

In [2]:
# Load each file into its own DataFrame
df_aprile = pd.read_excel(os.path.join(input_dir, "INDICATORI NETECH APRILE 25 (07 OTTOBRE 25).xlsx"))
df_giugno = pd.read_excel(os.path.join(input_dir, "INDICATORI NETECH GIUGNO 25 (07 OTTOBRE 25).xlsx"))
df_maggio = pd.read_excel(os.path.join(input_dir, "INDICATORI NETECH MAGGIO 25 (07 OTTOBRE 25).xlsx"))

# Print row counts before dropping duplicates
print(f"APRILE: {df_aprile.shape} rows before dropping duplicates")
print(f"GIUGNO: {df_giugno.shape} rows before dropping duplicates")
print(f"MAGGIO: {df_maggio.shape} rows before dropping duplicates")

# Drop duplicates for each
df_aprile = df_aprile.drop_duplicates()
df_maggio = df_maggio.drop_duplicates()
df_giugno = df_giugno.drop_duplicates()

# Print row counts after cleaning
print(f"APRILE: {df_aprile.shape} rows after dropping duplicates")
print(f"GIUGNO: {df_giugno.shape} rows after dropping duplicates")
print(f"MAGGIO: {df_maggio.shape} rows after dropping duplicates")

APRILE: (1016, 34) rows before dropping duplicates
GIUGNO: (1068, 34) rows before dropping duplicates
MAGGIO: (1094, 34) rows before dropping duplicates
APRILE: (1016, 34) rows after dropping duplicates
GIUGNO: (1068, 34) rows after dropping duplicates
MAGGIO: (1094, 34) rows after dropping duplicates


In [ ]:
# Stack them into one final DataFrame
df_NETECH = pd.concat([df_aprile, df_maggio, df_giugno], ignore_index=True)

print(f"\nFinal stacked DataFrame: {df_NETECH.shape} rows total")

output_file = os.path.join(output_dir, "NETECH_DATA_COMPLETTO.xlsx")
# Export DataFrame to Excel
df_NETECH.to_excel(output_file, index=False)

print(f"Exported df_NETECH to: {output_file}")


Final stacked DataFrame: (3178, 34) rows total
Exported df_NETECH to: 01_output\NETECH_DATA_COMPLETTO.xlsx


### LOAD OPEN AML DATA

In [5]:
openAML = "INDICATORI OPENAML (NETECH + SAD + OGGETTIVE) DATA ELABORAZIONE LUGLIO AGOSTO SETTEMBRE 25.xlsx"

df_AML_EVENTO = pd.read_excel(os.path.join(input_dir,openAML),
                              sheet_name="ELENCO PER EVENTO")

df_AML_POSIZIONE = pd.read_excel(os.path.join(input_dir,openAML),
                              sheet_name="ELENCO PER POSIZIONE")


print(df_AML_EVENTO.shape)
print(df_AML_POSIZIONE.shape)

(12863, 25)
(8509, 21)


### Adding needed data to NETECH

Add the stato from position level Open AML to NETECH

In [6]:
df_NETECH["ANNOMESE"] = df_NETECH["DATA OPERAZIONE"].astype(str).str[:6]

print(df_NETECH["ANNOMESE"].unique())
print(df_aprile["DATA OPERAZIONE"].astype(str).str[:6].unique())
print(df_maggio["DATA OPERAZIONE"].astype(str).str[:6].unique())
print(df_giugno["DATA OPERAZIONE"].astype(str).str[:6].unique())

['202504' 'nan' '202505' '202506']
['202504' 'nan']
['202505' 'nan']
['202506' 'nan']


In [7]:
print('########## Totale ##########')
print(df_NETECH["ANNOMESE"].value_counts(dropna=False))
print('########## Aprile ##########')
print(df_aprile["DATA OPERAZIONE"].astype(str).str[:6].value_counts(dropna=False))
print('########## Maggio ##########')
print(df_maggio["DATA OPERAZIONE"].astype(str).str[:6].value_counts(dropna=False))
print('########## Giugno ##########')
print(df_giugno["DATA OPERAZIONE"].astype(str).str[:6].value_counts(dropna=False))

########## Totale ##########
ANNOMESE
202505    1074
202506    1052
202504    1000
nan         52
Name: count, dtype: int64
########## Aprile ##########
DATA OPERAZIONE
202504    1000
nan         16
Name: count, dtype: int64
########## Maggio ##########
DATA OPERAZIONE
202505    1074
nan         20
Name: count, dtype: int64
########## Giugno ##########
DATA OPERAZIONE
202506    1052
nan         16
Name: count, dtype: int64


In [22]:
df_AML_POSIZIONE["ANNOMESE"] = (df_AML_POSIZIONE['Data elaborazione'] - pd.DateOffset(months=3)).dt.strftime("%Y%m")

df_merged_NETECH_1 = df_NETECH.merge(df_AML_POSIZIONE[["CDG", "Stato", "ANNOMESE"]].drop_duplicates(), how="left", left_on=["TITOLARE NDG", "ANNOMESE"], right_on=["CDG", "ANNOMESE"])

df_merged_NETECH_1 = df_merged_NETECH_1.drop(columns=["CDG"])

print(df_AML_POSIZIONE["ANNOMESE"].unique())
print(df_NETECH.shape)
print(df_merged_NETECH_1.shape)

print('Control the logic of months shift')
df_AML_POSIZIONE[['Data elaborazione', "ANNOMESE"]].drop_duplicates()

['202504' '202505' '202506']
(3178, 35)
(3178, 36)
Control the logic of months shift


,Data elaborazione,ANNOMESE
0,2025-07-01,202504
3089,2025-08-01,202505
6187,2025-09-01,202506


In [23]:
df_merged_NETECH_1['Stato'].value_counts(dropna=False)

Stato
IN VALUTAZIONE RPR           1182
ARCHIVIATA                    949
ARCHIVIATA DA RPR             503
IN CARICO AL GESTORE          297
NaN                           133
IN CARICO AL RESPONSABILE     112
CONFERMATA RPR                  1
RILEVAZIONE SAD                 1
Name: count, dtype: int64

In [24]:
stato_map = {"ARCHIVIATA": 0, "IN VALUTAZIONE RPR": 1, "ARCHIVIATA DA RPR": 0, 
             "IN CARICO AL GESTORE": 1, "IN CARICO AL RESPONSABILE": 1, "CONFERMATA RPR": 0,
             "ARCHIVIATA (2)": 0, "CONFERMANTA AML (3)": 0, "RILEVAZIONE SAD": 1,
             "CONFERMATA AML": 0, "CONFERMATA AML (2)": 0, "DA APPROFONDIRE RPR": 1,
             "DA APPROFONDIRE RESPONSABILE": 1, "DA INOLTRARE SOS": 2}

df_merged_NETECH_1["flag_stato"] = df_merged_NETECH_1["Stato"].map(stato_map)
df_merged_NETECH_1['flag_stato'].value_counts()

flag_stato
1.0    1592
0.0    1453
Name: count, dtype: int64

# Adding scenario in use of banks from Open AML Event level to NETECH

From ELENCO COMPLETO INDICATORI PROCEDURA OPENAML.xlsx, manualy create the list bellow for NETECH. it includes scenarios currently in use by banks

In [25]:
SCEN_IN_USO = ["14","30","40","83","90","95","100","DC01-00","DC03-01","DC04-00","DC05-00","DC06-02","DC07-02","DF01-02","DF02-08","DF03-02","DU01-01","DU01-03","DU01-05","DU01-06","DU01-07","DU01-10","DU01-16","PM-CS11","PM-CS20","PM-CS31","PM-CS50","PM-CS51","PM-SA11","PM-SA12","PM-SA20","PM-SA31","PM-SA33","PM-SA34","PM-SA53","PM-SA35","DU01-09","DC02-02"]
len(SCEN_IN_USO)

38

In [26]:
# exact-match flag
df_merged_NETECH_1["CODICE"] = df_merged_NETECH_1["CODICE"].astype(str)
df_merged_NETECH_1['flg_SCEN_IN_USO'] = df_merged_NETECH_1['CODICE'].isin(SCEN_IN_USO).astype(int)
print(df_merged_NETECH_1[df_merged_NETECH_1['flg_SCEN_IN_USO'] == 1].shape)

(3094, 38)


In [37]:
len(set(df_merged_NETECH_1["CODICE"].values))

12

In [ ]:
missing = set(df_merged_NETECH_1["CODICE"].values)-set(SCEN_IN_USO)
print("Scenario present in provided Netech data, but not in use:", missing)
print("Count:", len(missing))

Present in Netech data, not in use: {'25', '15', '24'}
Count: 3


In [36]:
missing = set(SCEN_IN_USO) - set(df_merged_NETECH_1["CODICE"].values)
print("Scenario present in in-use list, but not present in Netech perimeter:", missing)
print("Count:", len(missing))


Scenario present in in-use list, but not present in Netech perimeter: {'DU01-09', 'DC01-00', 'PM-SA34', 'DC04-00', 'PM-CS11', 'DU01-16', 'PM-SA33', 'DC03-01', 'PM-SA20', 'DU01-03', 'DU01-06', 'PM-SA12', 'DU01-01', 'PM-CS51', 'DC07-02', 'PM-CS50', 'PM-SA11', 'DF02-08', 'PM-CS20', 'PM-SA35', 'DC05-00', 'DC06-02', 'DU01-10', 'PM-SA53', 'DF01-02', 'DU01-07', 'DC02-02', 'DU01-05', 'PM-CS31'}
Count: 29


Creating the flag to mark the Oggettiva di valutare (was not used in the analisis)

In [38]:
df_filtered = df_AML_POSIZIONE[df_AML_POSIZIONE["Indicatore di anomalia"].str.contains("Oggettive da valutare", na=False)]
df_filtered['flag_OggValut'] = 1

df_merged_3 = df_merged_NETECH_1.merge(df_filtered[["CDG", "flag_OggValut","ANNOMESE", "Indicatore di anomalia"]].drop_duplicates(), how="left", left_on=["TITOLARE NDG", "ANNOMESE"], right_on=["CDG", "ANNOMESE"])

print(df_AML_POSIZIONE.shape)
print(df_filtered.shape)
print(df_merged_NETECH_1.shape)
print(df_merged_3.shape)
print(df_merged_3['flag_OggValut'].sum())

(8509, 22)
(3719, 23)
(3178, 38)
(3178, 41)
1015.0


C:\Users\sepehr.jafari\AppData\Local\Temp\ipykernel_26796\2941715944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['flag_OggValut'] = 1


Flag the residual causale (to identify sportello and VV causali; not used in the analysis)

In [41]:
# Ensure the column is string
df_merged_3["CAUSALE ANALITICA"] = df_merged_3["CAUSALE ANALITICA"].astype(str)

# Flag rows where CAUSALE ANALITICA is in eliminate_list
eliminate_list = ['451', '452', '14M', '56', 'ES', 'Y11', 'Y12', 'Y13', 'AFI', 'VV']
df_merged_3["flag_anom_causale"] = df_merged_3["CAUSALE ANALITICA"].isin(eliminate_list).astype(int)

print(df_merged_3["flag_anom_causale"].sum())


728


Create the output

In [42]:
# da eliminare quando flg_SCEN_IN_USO != 1
test1 = df_merged_3[df_merged_3['flg_SCEN_IN_USO'] == 1]
print(test1.shape)

# da eliminare quando flag_OggValut = 1
test2 = test1[test1['flag_OggValut'] != 1]
print(test2.shape)

# da eliminare quando flag_anom_causale == 1
test3 = test2[test2['flag_anom_causale'] != 1]
print(test3.shape)

(3094, 42)
(2088, 42)
(1655, 42)


In [43]:
len(set(test1['DESCRIZIONE']))

9

In [67]:
output_file = os.path.join(output_dir, "NETECH_tutti_scen_stato_v3.xlsx")
df_merged_3.to_excel(output_file, index=False)